In [1]:
# Imports
import IPython
import IPython.display
import pandas as pd
from numpy import mean
from numpy import std
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import glob
import tensorflow as tf
import math
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

np.random.seed(37151)
tf.random.set_seed(37151)

In [2]:
time = 2.0
steps = int(time * 200)

In [3]:
class LSTMClassification():
    def __init__(self):
         self.df = pd.read_csv('../master_df_normalized_36.csv')     
    
    # model_output_size = numero de categorias a determinar
    # mag_offset = valor magnitudeminimo a categorizar
    def prep_features(self, model_output_size=3, mag_offset=4, timedelta = 400):
        self.timedelta = timedelta
        self.model_output_size = model_output_size
        features = []
        labels = []
        for index, row in self.df.iterrows():
            try:
                records = row['list_records'].replace("'", '')
                records = records.replace('[', '')
                records = records.replace(']', '')                                  
                list_records = records.split(', ')
                magnitude = row['Mag']
                for record in list_records:
                    df_ts = pd.read_csv("../Renamed_Records/" + record, index_col=None, header=0, dtype=np.float32)
                    features.append(df_ts.iloc[:timedelta].to_numpy(dtype='float'))
                    labels.append(np.eye(model_output_size)[magnitude-mag_offset])
            except:
                continue
        self.features=np.array(features)
        self.labels=np.array(labels)
    
    def split_dataset(self, split = 0.8):
        self.trainX, self.testX, self.trainY, self.testY = train_test_split(self.features, self.labels, test_size = split, shuffle = True)
        print(self.trainX)
        print(self.trainY)
    
    def create_model(self, height=-1, width=2, model_output_size=-1):
        model = Sequential()
        if height == -1:
            height = self.timedelta
            
        if model_output_size == -1:
            model_output_size = self.model_output_size

        # input shape = num timesteps, num features(temp, acc.)
        model.add(LSTM(100, input_shape = (height, width)))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation = 'relu'))
        model.add(Dense(model_output_size, activation = 'softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Printing the models summary
        #model.summary()
        return model
    
    def evaluate_model(self, model, trainX, trainY, testX, testY, epochs=100, batch_size=3, verbose=0):
        history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
        loss, accuracy = model.evaluate(testX, testY, batch_size, verbose=verbose)
        return accuracy
    
    def predict_model(self, model, data):
        features = data['Values'].to_numpy().astype(float)
        predictions = model.predict(features)
        return predictions

In [4]:
# 2 segundos

classifier = LSTMClassification()
classifier.prep_features(model_output_size=5, mag_offset=3, timedelta = 2*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(180, 400, 2)
(180, 5)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -8.15772960e-09]
  [ 9.99999978e-03 -1.73351751e-08]
  ...
  [ 1.98500001e+00 -4.38477983e-08]
  [ 1.99000001e+00  1.40720843e-07]
  [ 1.99500000e+00  3.02855710e-07]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  0.00000000e+00]
  [ 9.99999978e-03  1.01971620e-09]
  ...
  [ 1.98500001e+00  0.00000000e+00]
  [ 1.99000001e+00  0.00000000e+00]
  [ 1.99500000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  1.32563107e-08]
  [ 9.99999978e-03  2.65126214e-08]
  ...
  [ 1.98500001e+00 -2.51869892e-07]
  [ 1.99000001e+00 -3.72196411e-07]
  [ 1.99500000e+00 -4.32359684e-07]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  7.44596764e-06]
  [ 9.99999978e-03  1.48185163e-05]
  ...
  [ 1.98500001e+00  6.08487113e-04]
  [ 1.99000001e+00  2.18561894e-04]
  [ 1.99500000e+00 -2.24723015e-04]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  4.07886480e-09]
  [

0.1736111044883728

In [5]:
# 4 segundos

classifier = LSTMClassification()
classifier.prep_features(model_output_size=5, mag_offset=3, timedelta = 4*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(180, 800, 2)
(180, 5)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  2.24337562e-08]
  [ 9.99999978e-03  4.48675124e-08]
  ...
  [ 3.98499990e+00 -5.09858111e-08]
  [ 3.99000001e+00 -3.35486646e-07]
  [ 3.99499989e+00 -2.44731893e-07]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  5.49208962e-05]
  [ 9.99999978e-03  1.09901957e-04]
  ...
  [ 3.98499990e+00  1.12160423e-03]
  [ 3.99000001e+00  1.31289475e-04]
  [ 3.99499989e+00 -6.14704331e-04]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -2.54929056e-08]
  [ 9.99999978e-03 -5.09858111e-08]
  ...
  [ 3.98499990e+00  4.01768176e-07]
  [ 3.99000001e+00  5.19035552e-07]
  [ 3.99499989e+00  5.00680642e-07]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -2.03943240e-09]
  [ 9.99999978e-03 -4.07886480e-09]
  ...
  [ 3.98499990e+00 -5.29946510e-06]
  [ 3.99000001e+00 -2.44731882e-06]
  [ 3.99499989e+00  1.43576040e-06]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  3.36506361e-08]
  [

0.1736111044883728